# Filter dataset

In [2]:
import codecs
import re

import numpy as np
import pandas as pd
import spacy
from tqdm import tqdm

In [3]:
nlp = spacy.load('en')

In [4]:
docs_index = pd.read_csv('../../data/documents.csv')
questions = pd.read_csv('../../data/qaps.csv')

In [5]:
docs_index = docs_index.loc[lambda df: df['kind'] == 'gutenberg']
questions = questions.loc[lambda df: df['question'].str.startswith('Who')]
questions = questions.merge(docs_index.loc[:, ['document_id']], how='inner', on='document_id')

In [6]:
docs_index.head()

,document_id,set,kind,story_url,story_file_size,wiki_url,wiki_title,story_word_count,story_start,story_end
1,0029bdbe75423337b551e42bb31f9a102785376f,train,gutenberg,http://www.gutenberg.org/ebooks/21572.txt.utf-8,814507,http://en.wikipedia.org/wiki/Percival_Keene,Percival Keene,173334,Produced by Nick,new eBooks .
2,00936497f5884881f1df23f4834f6739552cee8b,train,gutenberg,http://www.gutenberg.org/ebooks/3526.txt.utf-8,566874,http://en.wikipedia.org/wiki/Five_Weeks_in_a_B...,Five Weeks in a Balloon,112898,Produced by Judy,new eBooks .
3,00950a3641e6a28b04a6fabf6334140e2deaa9fd,train,gutenberg,http://www.gutenberg.org/ebooks/42188.txt.utf-8,90192,http://en.wikipedia.org/wiki/Shadows_in_the_Mo...,Shadows in the Moonlight (story),17670,Produced by Greg,new eBooks .
6,00fb61fa7bee266ad995e52190ebb73606b60b70,valid,gutenberg,http://www.gutenberg.org/ebooks/3771.txt.utf-8,372868,http://en.wikipedia.org/wiki/Cynthia's_Revels,Cynthia's Revels,74928,Produced by Sue,new eBooks .
7,014de1a8802c05ff64efa047e9290fb7fccea2b4,test,gutenberg,http://www.gutenberg.org/ebooks/1329.txt.utf-8,560685,http://en.wikipedia.org/wiki/A_Voyage_to_Arcturus,A Voyage to Arcturus,113790,Produced by An,new eBooks .


In [7]:
questions.head()

,document_id,set,question,answer1,answer2,question_tokenized,answer1_tokenized,answer2_tokenized
0,0029bdbe75423337b551e42bb31f9a102785376f,train,Who is Miss Delmer?,the elderly spinster aunt of the Earl de Verse...,She's Captail Delmar's aunt.,Who is Miss Delmer ?,the elderly spinster aunt of the Earl de Verse...,She s Captail Delmar s aunt .
1,0029bdbe75423337b551e42bb31f9a102785376f,train,Who does Arabella Mason wed?,"Ben Keene, Delmar's valet",Ben Keene,Who does Arabella Mason wed ?,"Ben Keene , Delmar s valet",Ben Keene
2,0029bdbe75423337b551e42bb31f9a102785376f,train,Who is the bully that steals Percival's lunch?,"his teacher, Mr. O'Gallagher",The schoolmaster,Who is the bully that steals Percival s lunch ?,"his teacher , Mr. O'Gallagher",The schoolmaster
3,0029bdbe75423337b551e42bb31f9a102785376f,train,Who does Percival convince the Pirates to spare?,a rich Dutch merchant and his daughter Minnie,A Dutch Merchant and his daughter,Who does Percival convince the Pirates to spare ?,a rich Dutch merchant and his daughter Minnie,A Dutch Merchant and his daughter
4,0029bdbe75423337b551e42bb31f9a102785376f,train,Who lives at Madeline Hall?,Miss Delmar,Miss Delmar,Who lives at Madeline Hall ?,Miss Delmar,Miss Delmar


In [9]:
docs_index.to_csv('../../data/documents_books.csv', index=False)
questions.to_csv('../../data/questions_books_who.csv', index=False)